Формулировка задачи:

Необходимо создать консольную (работающую из командной строки) утилиту. Программа спрашивает домен и забирает с введённого адреса файл robots.txt. Затем скрипт парсит файл и выводит его содержимое в виде объекта. Ключами объекта являются параметры User-Agent, а значениями — вложенный объект. Вложенный объект содержит два поля Allow и Dissallow, каждый из которых является массивом соответствующих URL из robots.txt. Код выложить на GitHub
Пример:

{
  "*": {
      "Disallow": ["/cgi-bin"],
      "Allow": ["/"]
   },
  "GoogleBot": {
      "Disallow": ["/cgi-bin"],
      "Allow": ["/"]
   }
}

Декомпозиция задачи:
1. У нас консольная программа, но передавать параметры при запуске вроде бы не нужно. Есть слово "скрипт", но нет ограничения на язык. Пусть будет python, его (при прочих равных) приятно читать и видеть. Системой пусть будет linux, но ничего платформоспецифичного вроде бы не предполагается.

2. Нужно спросить о домене и забрать с введённого адреса. Предполагаю, что "домен" и "адрес" здесь понимаются как синонимы, а не как "домен + адрес" или как-то ещё. Есть два способа это сделать - написать текстом строку и вбить IP-адрес. Можно поставить проверки на основе регулярных выражений, которые покажут, является ли введённый текст чем-то осмысленным. Ещё можно ругаться, если нет такого адреса, но для этого вроде бы есть готовая ругалка

3. Нужно вытащить файл и прочитать. Возможные проблемы: по адресу нет файла, нет нужной структуры внутри.

4. Чтобы вернуть что нужно, можно было бы определить пользовательский объект. Но вид объекта наводит на мысль либо о словаре словарей, который уже реализован. Воспользуемся им.

5. В требованиях есть слово "выводит содержимое в виде объекта". По некоторому размышлению, предполагает, что мы, возможно, хотим уметь сериализовать (или хотя бы выводить на печать) полученный объект, чтобы прямо в консоли видеть результат. С этим тоже нет проблемы, но для красивого вывода можно пробежаться по ключам, чтобы не смотреть совсем уж на мешанину.

Методы:

1. Получить url
2. Соединиться
3. Распарсить
4. Прибраться за собой


Замечу, что на stackoverflow <a href = https://stackoverflow.com/questions/43085744/parsing-robots-txt-in-python#43086135> есть </a> наводящее на нужные мысли (правда, для python 2) решение такого вот вида:

In [1]:
import os
result = os.popen("curl https://fortune.com/robots.txt").read()
result_data_set = {"Disallowed":[], "Allowed":[]}

for line in result.split("\n"):
    if line.startswith('Allow'):    # this is for allowed url
        result_data_set["Allowed"].append(line.split(': ')[1].split(' ')[0])    # to neglect the comments or other junk info
    elif line.startswith('Disallow'):    # this is for disallowed url
        result_data_set["Disallowed"].append(line.split(': ')[1].split(' ')[0])    # to neglect the comments or other junk info

print (result_data_set)

{'Allowed': ['/wp-admin/admin-ajax.php'], 'Disallowed': ['/wp-admin/', '/search/', '/wp-login.php', '/activate/', '/cgi-bin/', '/mshots/v1/', '/next/', '/public.api/']}


Кроме того, оно не учитывает наличие нескольких user-agent.

Дополним его и избавим от указанных проблем, попутно придав формат тестового задания.

In [190]:
import requests
import os
import re


def is_string_a_valid_url(string):
    """
    This function add some validation for our user input. This one is for the case when we expect a url to be entered.
    The idea of the regular expression is from here: https://web.izjum.com/regexp-email-url-phone (in Russian)
    :param string: a string we want to check
    :return: True or False
    """
    url_regexp = '^((https?|ftp)\:\/\/)?([a-z0-9]{1})((\.[a-z0-9-])|([a-z0-9-]))*\.([a-z]{2,6})(\/?)$'
    if re.match(url_regexp, string):
        return True
    else:
        return False


def ask_for_url():
    """
    The first requirement for this was to read a user's input.
    Using previously defined functions, namely: is_string_a_valid_url() and is_string_a_valid_ip()
    we are going to make a simple sanity check.

    It also seems convenient to automatically add an 'https://' prefix if there is no protocol specified.
    and add '/robots.txt'

    :return: a url if everything went well. Returns 1 in case of errors.
    """
    url_entered = input("Enter the url you wich to retrieve 'robots.txt' from: ")

    if not url_entered.startswith('http') and not url_entered[0].isdigit():
        url_entered = "https://" + url_entered

    if is_string_a_valid_url(url_entered) and not url_entered.endswith('/robots.txt'):
        url_entered += '/robots.txt'
        return url_entered
    else:
        print("You're probably entering not a valid url. Please make sure the input is correct.")
        return 1


def download_robots_txt(path_to_robots_txt, my_user_agent='*'):
    """
    This method uses the 'request' package to retrieve a file from the url specified.
    An optional parameter is for the (unlikely) case where we need a specific user-agent to be able to download
    a file we want.

    It uses a procedure that iterates over a file we download (for almost impossible case of robots.txt
    being exceptionally large).

    Prints out the logstring and the filename, just for convenience

    :param path_to_robots_txt: a full path to robots.txt.
    :param my_user_agent: in case we need to specify one
    :return:
    """
    local_filename = "robots_local_temp.txt"
    my_headers = requests.utils.default_headers()
    my_headers.update({'User-Agent': my_user_agent})
    try:
        r = requests.get(path_to_robots_txt, headers=my_headers)
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:  # filter out keep-alive new chunks
                    f.write(chunk)
        logstring = 'download finished, temporary file ./' + local_filename + ' created'
        print(logstring)
        return 0
    except requests.exceptions.RequestException as e:
        print(e)


def parse_robots_txt():
    """
    Parses a local copy of the file 'robots.txt' we probably have.

    To pass a file only once, we needed a logic that allows to store the very first user-agent name as a key,
    and associate the collected data with it only after we have met the next one (or the end of file).

    :return: a resulting data structure which is essentially a dictionary of dictionaries. The external keys are
    User-Agents, the internal ones are 'Allowed' and 'Disallowed'. The internal values are lists of domains
    """
    if os.path.exists("robots_local_temp.txt"):
        local_file = open('robots_local_temp.txt', 'r')
        resulting_data_set = {}
        agent_name_current = 'NoAgent'
        agent_name_next = 'NoAgent'
        for line in local_file:
            line = line.split('\n')[0]
            if line.startswith('User-agent'):
                agent_name_next = line.split(': ')[1].split(' ')[0]
                if not agent_name_current == 'NoAgent':
                    resulting_data_set[agent_name_current] = data_set_for_current_user_agent
                agent_name_current = agent_name_next
                data_set_for_current_user_agent = {"Disallowed": [], "Allowed": []}
            else:
                if line.startswith('Allow'):
                    # filter some possibly useless info and comments
                    data_set_for_current_user_agent["Allowed"].append(line.split(': ')[1].split(' ')[0])
                elif line.startswith('Disallow'):
                    # same thing here
                    data_set_for_current_user_agent["Disallowed"].append(line.split(': ')[1].split(' ')[0])
        # we now have the info of the last (and probably the unique one) agent. So it is time to write them to a
        # data structure we have and stop.
        agent_name_current = agent_name_next
        resulting_data_set[agent_name_current] = data_set_for_current_user_agent
        return resulting_data_set
    else:
        print('A local copy of robots.txt is missing for some reason.')
        return 1


def print_out_the_result(result):
    """
    This function is to generate a bit more human-readable form of the output.
    Makes a bit easier to track which 'Allows' and 'Disallows' suits a User-Agent we are interested in.
    :param: a data structure returned by parse_robots_txt() method
    :return: 0 - for and ideal case. 1 - if we have no output but somehow reached this point and launched a function
    """
    if result == 1:
        print("No output created")
        return 1
    else:
        for key in result.keys():
            print(key, ':')
            for inner_key in result[key]:
                print(inner_key, ':')
                print(result[key][inner_key])
            print()
        return 0


def dispose_of_temp_file():
    """
    One way to parse a file is to download it and store its local version.
    However, it is quite awkward to leave a user with a file he didn't want to create.
    Or use a file with the same name someone occasionally left there for some reason.
    So, before and after we run our code, we need to clean up.

    :return: 0 - if we can remove a file. Raises a warning otherwise.
    """
    if os.path.exists("robots_local_temp.txt"):
        try:
            os.remove("robots_local_temp.txt")
            print('cleaning up...')
        except OSError:
            raise UserWarning("Cannot remove 'robots_local_temp.txt'. Is it opened somewhere?")
    return 0


In [183]:
dispose_of_temp_file()

cleaning up...


0

In [184]:
url = ask_for_url()
print(url)

Enter the url you wich to retrieve 'robots.txt' from: habrahabr.ru
https://habrahabr.ru/robots.txt


In [185]:
download_robots_txt(url)

download finished, temporary file ./robots_local_temp.txt created


0

In [186]:
result_obtained = parse_robots_txt()
print_out_the_result(result_obtained)

{'Yandex': {'Allowed': [], 'Disallowed': ['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']}, '*': {'Allowed': [], 'Disallowed': ['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']}, 'Slurp': {'Allowed': [], 'Disallowed': ['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']}, 'Googlebot': {'Allowed': [], 'Disallowed': ['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']}}


In [191]:
dispose_of_temp_file()

Yandex :
Allowed :
[]
Disallowed :
['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']

* :
Allowed :
[]
Disallowed :
['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']

Slurp :
Allowed :
[]
Disallowed :
['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']

Googlebot :
Allowed :
[]
Disallowed :
['/search/', '/js/', '/css/', '/ajax/', '/login/', '/register/', '/*utm_']



0